In [2]:
import pandas as pd 
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException
import time 
import re 
from bs4 import BeautifulSoup 
from tqdm import tqdm 

In [3]:
# css_selector 체크 함수 
def check_exists_by_css(driver, text):
    try:
        driver.find_element_by_css_selector(text)
    except NoSuchElementException:
        return False
    return True

### 서울 지역별 맛집 이름 가져오기

In [ ]:
driver = webdriver.Chrome('/Users/sol/Downloads/chromedriver')

keyword = ['강서구 맛집', '양천구 맛집', '구로구 맛집', '금천구 맛집', '영등포구 맛집', '동작구 맛집', '관악구 맛집', '서초구 맛집', '강남구 맛집', '송파구 맛집', '강동구 맛집',
          '마포구 맛집', '용산구 맛집', '성동구 맛집', '광진구 맛집', '서대문구 맛집', '중구 맛집', '동대문구 맛집', '중랑구 맛집', '성북구 맛집', '은평구 맛집', '강북구 맛집', '도붕구 맛집', '노원구 맛집']

def scrapping(keyword):
    word_list = []
    df = pd.DataFrame()
    naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place'
    driver.get(naver_map_search_url)
    time.sleep(2)
    driver.switch_to.frame('searchIframe')
    time.sleep(2)

    for i in range(1,7):
        click = driver.find_element_by_xpath(f'//*[@id="app-root"]/div/div[2]/div[2]/a[{i}]')
        click.click()

        scroll_div = driver.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[1]")
        driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
        time.sleep(2)
        driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
        time.sleep(2)
        driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
        time.sleep(2)
        driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
        time.sleep(2)
        driver.execute_script("arguments[0].scrollBy(0,2000)", scroll_div)
        time.sleep(2)

        words = driver.find_elements_by_class_name('OXiLu')
        word = [word.text for word in words]
        word_list += word
            
    df['title'] = word_list
    df['keyword'] = keyword

    return df 

In [ ]:
full = pd.DataFrame()

for i in range(len(keyword)):
    test = scrapping(keyword[i])
    full = pd.concat([full, test], axis=0)

### 맛집 이름으로 url 가져오기

In [ ]:
full = full.drop_duplicates()
full = full.reset_index(drop=True)

full['naver_map_url'] = ''
title = full['keyword'] + full['title']
title[3397] = '성동구 맛집봉순이네다락방'

In [ ]:
for i, keyword in enumerate(title.tolist()): 
    print("이번에 찾을 키워드 :", i, f"/ {title.shape[0]} 행", keyword) 
    try: 
        naver_map_search_url = f'https://map.naver.com/v5/search/{keyword}/place' # 검색 url 만들기 
        driver.get(naver_map_search_url) # 검색 url 접속, 즉 검색하기 
        time.sleep(3)
        driver.switch_to.frame('searchIframe')
        time.sleep(3) 
        
        try:
            click = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/ul/li[1]/div[2]/a[1]/div/div/span[1]')
            click.click()
        except:
            pass
        
        cu = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지 
        res_code = re.findall(r"place/(\d+)", cu) 
        final_url = 'https://pcmap.place.naver.com/restaurant/'+res_code[0]+'/review/visitor#' 
        print(final_url) 
        full['naver_map_url'][i]=final_url 
        
    except IndexError: 
        full['naver_map_url'][i]= '' 
        print('none') 
#         test.to_csv('url_completed.csv', encoding = 'utf-8-sig')

### 가져온 url로 식당이름, 주소, 카테고리 가져오기 (이름과 주소는 크로스 체크를 위해서)

In [33]:
driver = webdriver.Chrome('/Users/sol/Downloads/chromedriver')

In [5]:
full = pd.read_csv('full.csv')

In [6]:
url = full['naver_map_url'].apply(lambda x:x.split('review')[0])
full['naver_home_url'] = url.apply(lambda x:x+'home')

In [70]:
name_list = []
address_list = []
restaurant_classificaton_list = []

for i in range(len(full['naver_home_url'].tolist())):
    first_url = full['naver_home_url'][i]
    driver.get(first_url) 
    time.sleep(1)
    name = driver.find_elements_by_class_name('_3XamX')[0].text
    time.sleep(1)
    address = driver.find_elements_by_class_name('_2yqUQ')[0].text
    time.sleep(1)
    restaurant_classificaton= driver.find_elements_by_class_name('_3ocDE')[0].text
    time.sleep(1)
    
    name_list.append(name)
    address_list.append(address)
    restaurant_classificaton_list.append(restaurant_classificaton)  

In [74]:
full['new_title'] = name_list
full['address'] = address_list
full['category'] = restaurant_classificaton_list

### 평점, 아이디, 리뷰 가져오기

In [158]:
driver = webdriver.Chrome('/Users/sol/Downloads/chromedriver')

In [86]:
test = pd.read_csv('half_3888.csv')

In [73]:
# href 체크 함수 
def check_exists_href(soup_list, i, element, class_name):
    try:
        soup_list[i].find('div', attrs = {'class':class_name}).find('a').get('href') 
    except:
        return False
    return True

# text 체크 함수 
def check_exists_by_text(soup_list, i, element, class_name):
    try:
        soup_list[i].find(element, attrs = {'class':class_name}).text  
    except:
        return False
    return True

In [169]:
# 수집할 정보들 
rating_list = [] # 평점 
user_review_id = {} # 유저의 id - 추천시스템에서는 필수 
review_json = {} # 리뷰 


for i in range(2005, 2500): 
    print('======================================================') 
    print(str(i)+'번째 식당') 
    # 식당 리뷰 개별 url 접속 
    driver.get(test['naver_map_url'][i]) 
    thisurl = test['naver_map_url'][i] 
    time.sleep(1)
    if check_exists_by_css(driver, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._1fQmI > h2 > a'):
        driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._1fQmI > h2 > a').click()
    else:
        pass


    while True:  
        driver.find_element_by_tag_name('body').send_keys(Keys.END) 
        time.sleep(1) 
        if check_exists_by_css(driver,'#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._1fQmI > div._2kAri > a'):
            driver.find_element_by_css_selector('#app-root > div > div > div > div:nth-child(7) > div:nth-child(2) > div.place_section._1fQmI > div._2kAri > a').send_keys(Keys.ENTER) 
            time.sleep(1) 
        else:
            break
    
    time.sleep(1)
    
    #파싱 
    html = driver.page_source 
    soup = BeautifulSoup(html, 'lxml') 
    time.sleep(1) 
    
    # 식당 구분 
    restaurant_name = test['title'][i] 
    print('식당 이름 : '+restaurant_name)
    
    user_review_id[restaurant_name] = {} 
    review_json[restaurant_name] = {}
    
    one_review = soup.find_all('li', attrs = {'class':'_3FaRE'}) 
    review_num = len(one_review) # 특정 식당의 리뷰 총 개수 
    print (review_num)
    
    # 리뷰 개수 
    for i in range(len(one_review)): 
        # user url 
        if check_exists_href(one_review, i, 'div', '_2h9Q5'):
            user_url = one_review[i].find('div', attrs = {'class':'_2h9Q5'}).find('a').get('href') 
            user_code = re.findall(r"my/(\w+)", user_url)[0] 
#             print('user_code = '+user_code) 
            # review 1개에 대한 id 만들기 
            res_code = re.findall(r"restaurant/(\d+)", thisurl)[0] 
            review_id = str(res_code)+"_"+user_code 
#             print('review_id = '+review_id) 
        else:
            user_url = ''
            review_id = ''
            
        # rating, 별점
        if check_exists_by_text(one_review, i, 'span', '_1fvo3 Sv1wj'):
            rating = one_review[i].find('span', attrs = {'class':'_1fvo3 Sv1wj'}).text 
#             print('rating = '+rating)
        else:
            rating = ''
        
#         # review, 리뷰
#         if check_exists_by_text(one_review, i, 'span', 'WoYOw'):
#             review_content = one_review[i].find('span', attrs = {'class' : 'WoYOw'}).text
#             print('review : done')
#         else:
#             review = ""
            
        naver_review = user_url, review_id, restaurant_name, rating
        rating_list.append(naver_review)

2005번째 식당
식당 이름 : 청와옥 석촌호수직영점
1130
2006번째 식당
식당 이름 : 마싯내
418
2007번째 식당
식당 이름 : 카페 노티드 잠실
5157
2008번째 식당
식당 이름 : 토닭토닭
516
2009번째 식당
식당 이름 : 코벤트가든 올림픽공원점
485
2010번째 식당
식당 이름 : 엘리스리틀이태리
319
2011번째 식당
식당 이름 : 등촌샤브칼국수 방이본점
1035
2012번째 식당
식당 이름 : 초대창 잠실새내점
94
2013번째 식당
식당 이름 : 명인밥상
434
2014번째 식당
식당 이름 : 동경산책
488
2015번째 식당
식당 이름 : 미엔아이
722
2016번째 식당
식당 이름 : 골목떡볶이
1369
2017번째 식당
식당 이름 : 벽제갈비 방이본점
404
2018번째 식당
식당 이름 : 서울리즘
528
2019번째 식당
식당 이름 : 아그라 잠실에비뉴엘점
745
2020번째 식당
식당 이름 : 꼬앙드파리
147
2021번째 식당
식당 이름 : 사시미 신천본점
114
2022번째 식당
식당 이름 : 일일향 잠실점
535
2023번째 식당
식당 이름 : 오복수산 잠실점
434
2024번째 식당
식당 이름 : 오레노라멘송파
707
2025번째 식당
식당 이름 : 고도식
465
2026번째 식당
식당 이름 : 스웨덴피크닉 송리단길
189
2027번째 식당
식당 이름 : 호이안로스터리 송리단길점
455
2028번째 식당
식당 이름 : 골드참치
135
2029번째 식당
식당 이름 : 우드그린4717
103
2030번째 식당
식당 이름 : 피자네버슬립스
364
2031번째 식당
식당 이름 : 담은갈비
327
2032번째 식당
식당 이름 : STAY
324
2033번째 식당
식당 이름 : Chitchat 칫챗
344
2034번째 식당
식당 이름 : 서보
31
2035번째 식당
식당 이름 : 바티칸
39
2036번째 식당
식당 이름 : 잠수교집 3호점
233
2037번째 식당
식당 이름 : 뉴질랜드스토리
385
2038번째 식당
식

식당 이름 : 프레스커피
514
2102번째 식당
식당 이름 : 도쿄등심 롯데캐슬잠실점
492
2103번째 식당
식당 이름 : 사케쇼프
57
2104번째 식당
식당 이름 : 골든타이
227
2105번째 식당
식당 이름 : 제일곱창 방이직영점
51
2106번째 식당
식당 이름 : 본가진미간장게장
230
2107번째 식당
식당 이름 : 쥬뗑뷔뜨
236
2108번째 식당
식당 이름 : 인기명 방이점
98
2109번째 식당
식당 이름 : 고쿠텐 잠실새내점
256
2110번째 식당
식당 이름 : 별미곱창
499
2111번째 식당
식당 이름 : 에이엠에이 커피로스터스
37
2112번째 식당
식당 이름 : 부일갈매기
126
2113번째 식당
식당 이름 : 또봄
172
2114번째 식당
식당 이름 : 카페마마스 잠실점
2026
2115번째 식당
식당 이름 : MIP
159
2116번째 식당
식당 이름 : 옥봉갈비
27
2117번째 식당
식당 이름 : 선호 커피 로스터스
97
2118번째 식당
식당 이름 : 이성원쉐프의 청년감자탕 본점
618
2119번째 식당
식당 이름 : 버들골이야기 방이점
152
2120번째 식당
식당 이름 : 수제맥주 바이더레이크
173
2121번째 식당
식당 이름 : 게장마을
39
2122번째 식당
식당 이름 : 야끼소바 니주마루
234
2123번째 식당
식당 이름 : 할머니포장마차멸치국수
541
2124번째 식당
식당 이름 : 인쌩맥주 잠실신천점
125
2125번째 식당
식당 이름 : 브루클린더버거조인트 롯데월드몰점
933
2126번째 식당
식당 이름 : 평이담백 뼈칼국수 방이점
381
2127번째 식당
식당 이름 : 형제화로
20
2128번째 식당
식당 이름 : 더 이탈리안 클럽
61
2129번째 식당
식당 이름 : 잠실장어와 한우
216
2130번째 식당
식당 이름 : 글로리식당
104
2131번째 식당
식당 이름 : 맥주유소
99
2132번째 식당
식당 이름 : 디트로네 라운지
418
2133번째 식당
식당 이름 : 도삭면공방
295
2134번

식당 이름 : 스테이지28
536
2198번째 식당
식당 이름 : 브랜뉴하이몬드
2907
2199번째 식당
식당 이름 : 차이나린찐
922
2200번째 식당
식당 이름 : 풍년상회
683
2201번째 식당
식당 이름 : 송하정스시
63
2202번째 식당
식당 이름 : 꿀꿀진순대 강동본점
520
2203번째 식당
식당 이름 : 미드데이썬
117
2204번째 식당
식당 이름 : 장어집 본점
257
2205번째 식당
식당 이름 : 스시마에
66
2206번째 식당
식당 이름 : 시에나블루 케이크
21
2207번째 식당
식당 이름 : 멘야세븐
358
2208번째 식당
식당 이름 : 쿠우쿠우 천호점
228
2209번째 식당
식당 이름 : 경양미식
95
2210번째 식당
식당 이름 : 평상시
177
2211번째 식당
식당 이름 : YUP
124
2212번째 식당
식당 이름 : 대가돈가츠
127
2213번째 식당
식당 이름 : 오늘손칼국수&찜 강동본점
891
2214번째 식당
식당 이름 : 등갈비달인 본점
291
2215번째 식당
식당 이름 : 함흥본가면옥
891
2216번째 식당
식당 이름 : 고모네원조콩탕황태탕
390
2217번째 식당
식당 이름 : 너츠커피바
20
2218번째 식당
식당 이름 : 와고참숯갈비
147
2219번째 식당
식당 이름 : 온전히
205
2220번째 식당
식당 이름 : 말뚝곱창 천호점
63
2221번째 식당
식당 이름 : 블랑제리11-17
1176
2222번째 식당
식당 이름 : 33Acre
93
2223번째 식당
식당 이름 : 씨푸드박스
149
2224번째 식당
식당 이름 : 최냉면 본점
310
2225번째 식당
식당 이름 : 안녕식당
884
2226번째 식당
식당 이름 : 시집그릴하우스
345
2227번째 식당
식당 이름 : 8번가
116
2228번째 식당
식당 이름 : 메이탄
336
2229번째 식당
식당 이름 : WAAAH
127
2230번째 식당
식당 이름 : 시오미스시
44
2231번째 식당
식당 이름 : 커피몽타주
862
2232번째

식당 이름 : 천호누룽지통닭구이 성내점
172
2296번째 식당
식당 이름 : 채선당 올림픽공원점
287
2297번째 식당
식당 이름 : 렛미베이크
97
2298번째 식당
식당 이름 : 스시야미 천호점
401
2299번째 식당
식당 이름 : 한성관
724
2300번째 식당
식당 이름 : 대패꽃필무렵 강동점
374
2301번째 식당
식당 이름 : 지리산왕재봉흑돼지
475
2302번째 식당
식당 이름 : 초밥의 신 부타
338
2303번째 식당
식당 이름 : 키친사계
130
2304번째 식당
식당 이름 : 바하마스
75
2305번째 식당
식당 이름 : 보타이
200
2306번째 식당
식당 이름 : 동구밖장작구이
258
2307번째 식당
식당 이름 : 그레인 제빵소
625
2308번째 식당
식당 이름 : 꾸러기분식
208
2309번째 식당
식당 이름 : 명일원조닭발
577
2310번째 식당
식당 이름 : 르뱅
11
2311번째 식당
식당 이름 : 선남sea
310
2312번째 식당
식당 이름 : 황도바지락칼국수
873
2313번째 식당
식당 이름 : 철원족발
471
2314번째 식당
식당 이름 : 만국미식회
80
2315번째 식당
식당 이름 : 김수환식당
207
2316번째 식당
식당 이름 : 어흥
34
2317번째 식당
식당 이름 : 맛쟁이떡볶이 강동역점
83
2318번째 식당
식당 이름 : 이자카야 인
114
2319번째 식당
식당 이름 : 숙성명가 함흥냉면 본점
196
2320번째 식당
식당 이름 : 소담촌 강동역점
419
2321번째 식당
식당 이름 : 지우팅
70
2322번째 식당
식당 이름 : 심플잇
33
2323번째 식당
식당 이름 : 청해진
285
2324번째 식당
식당 이름 : 잇다
167
2325번째 식당
식당 이름 : 파트원나이스
63
2326번째 식당
식당 이름 : 예가족발 본점
181
2327번째 식당
식당 이름 : 오마스시
87
2328번째 식당
식당 이름 : 핵밥 천호점
378
2329번째 식당
식당 이름 : 봉쥬르 쟝딸롱
63
2330번

식당 이름 : 부영각
282
2395번째 식당
식당 이름 : silhouette coffee
99
2396번째 식당
식당 이름 : 오이지 연남
49
2397번째 식당
식당 이름 : 옛맛서울불고기
594
2398번째 식당
식당 이름 : 툭툭누들타이
898
2399번째 식당
식당 이름 : 어반플랜트 합정
367
2400번째 식당
식당 이름 : 어글리베이커리
3617
2401번째 식당
식당 이름 : 서교주담
58
2402번째 식당
식당 이름 : 앤티크커피
77
2403번째 식당
식당 이름 : 샌드커피 논탄토 연남
487
2404번째 식당
식당 이름 : 서울페이스트리 연남점
206
2405번째 식당
식당 이름 : 오레노라멘 본점
1240
2406번째 식당
식당 이름 : 루나틱커피
97
2407번째 식당
식당 이름 : 아웃나우 홍대
39
2408번째 식당
식당 이름 : 몰라
117
2409번째 식당
식당 이름 : 나랑가
230
2410번째 식당
식당 이름 : 카쿠시타
193
2411번째 식당
식당 이름 : 홍대IN파스타
425
2412번째 식당
식당 이름 : 장작집
190
2413번째 식당
식당 이름 : 우동 이요이요
189
2414번째 식당
식당 이름 : 초이 다이닝
69
2415번째 식당
식당 이름 : 진미식당
531
2416번째 식당
식당 이름 : 을밀대 평양냉면
1646
2417번째 식당
식당 이름 : 와이키키마켓
333
2418번째 식당
식당 이름 : 마이클식당
135
2419번째 식당
식당 이름 : 연남토마
1026
2420번째 식당
식당 이름 : 위아소울메이트
308
2421번째 식당
식당 이름 : 조선초가한끼
249
2422번째 식당
식당 이름 : 우동 카덴
853
2423번째 식당
식당 이름 : 소금집델리 망원
1178
2424번째 식당
식당 이름 : 아오이토리
1241
2425번째 식당
식당 이름 : 불떡클럽 연남본점
208
2426번째 식당
식당 이름 : 역전회관 마포본점
828
2427번째 식당
식당 이름 : 경양카츠 연남점
518
2428번째 식

식당 이름 : 슬로우캘리 연남본점
909
2493번째 식당
식당 이름 : 산더미불고기
312
2494번째 식당
식당 이름 : 히츠지야 연남점
219
2495번째 식당
식당 이름 : 멕시칸식당 베르데
401
2496번째 식당
식당 이름 : 상암 구름산추어탕
450
2497번째 식당
식당 이름 : 카밀로 라자네리아
267
2498번째 식당
식당 이름 : 또보겠지떡볶이집 붕붕허니비점
899
2499번째 식당
식당 이름 : 저스트텐동 연남본점
339


In [172]:
rating_df = pd.DataFrame(rating_list) 
rating_df.columns = ['user_url','UserID','ItemID','Rating'] 
rating_df = rating_df[rating_df['Rating'] != '']

In [161]:
rating_df['ResCode'] = rating_df['UserID'].apply(lambda x:x.split('_')[0])
rating_df['UserID'] = rating_df['UserID'].apply(lambda x:x.split('_')[1])
rating_df['Rating'] = rating_df['Rating'].apply(lambda x:x.split('별점')[1])

In [118]:
rating_df.to_csv('done.csv', index=False)

In [123]:
concat = pd.concat([tmp,tmp2])

In [124]:
concat['ResCode'] = concat['UserID'].apply(lambda x:x.split('_')[0])
concat['UserID'] = concat['UserID'].apply(lambda x:x.split('_')[1])
concat['Rating'] = concat['Rating'].apply(lambda x:x.split('별점')[1])

In [127]:
concat = concat.drop_duplicates()

In [130]:
a = pd.read_csv('rating_done2.csv')

In [133]:
a['ItemID'].nunique()

1256

In [134]:
b = pd.read_csv('concat.csv')

In [135]:
b['ItemID'].nunique()

457

In [139]:
b

,user_url,UserID,ItemID,Rating,ResCode
0,https://m.place.naver.com/my/5e1877e68f87a842b...,5e1877e68f87a842bc8a03ce,오늘와인한잔 양천향교역점,3.0,1049678083
1,https://m.place.naver.com/my/5efdf404854ca9293...,5efdf404854ca9293112e92c,오늘와인한잔 양천향교역점,5.0,1049678083
2,https://m.place.naver.com/my/5f44cb53a602e98f5...,5f44cb53a602e98f564c63ad,오늘와인한잔 양천향교역점,3.0,1049678083
3,https://m.place.naver.com/my/5dd91a419ec8258e4...,5dd91a419ec8258e4a4342bb,오늘와인한잔 양천향교역점,5.0,1049678083
4,https://m.place.naver.com/my/5efde995b1f7027e8...,5efde995b1f7027e8a6ca104,오늘와인한잔 양천향교역점,3.0,1049678083
...,...,...,...,...,...
130202,https://m.place.naver.com/my/5f1d0b229ec8258e4...,5f1d0b229ec8258e4a4d47e7,봉평메밀막국수 본점,4.0,1295508661
130203,https://m.place.naver.com/my/5e82fd178f87a842b...,5e82fd178f87a842bca79e88,봉평메밀막국수 본점,5.0,1295508661
130204,https://m.place.naver.com/my/5c8392747faa6146a...,5c8392747faa6146a8713315,봉평메밀막국수 본점,0.5,1295508661
130205,https://m.place.naver.com/my/5effe99dfe03e278d...,5effe99dfe03e278d38b6f9a,봉평메밀막국수 본점,3.5,1295508661


In [136]:
c = pd.read_csv('rating_done1.csv')

In [137]:
c['ItemID'].nunique()

347

In [138]:
c

,user_url,UserID,ItemID,Rating
0,https://m.place.naver.com/my/5e1877e68f87a842b...,1049678083_5e1877e68f87a842bc8a03ce,오늘와인한잔 양천향교역점,별점3
1,https://m.place.naver.com/my/5efdf404854ca9293...,1049678083_5efdf404854ca9293112e92c,오늘와인한잔 양천향교역점,별점5
2,https://m.place.naver.com/my/5f44cb53a602e98f5...,1049678083_5f44cb53a602e98f564c63ad,오늘와인한잔 양천향교역점,별점3
3,https://m.place.naver.com/my/5dd91a419ec8258e4...,1049678083_5dd91a419ec8258e4a4342bb,오늘와인한잔 양천향교역점,별점5
4,https://m.place.naver.com/my/5efde995b1f7027e8...,1049678083_5efde995b1f7027e8a6ca104,오늘와인한잔 양천향교역점,별점3
...,...,...,...,...
97850,https://m.place.naver.com/my/5e084daf8f87a842b...,1671040944_5e084daf8f87a842bca8ff24,배꼽집,별점5
97851,https://m.place.naver.com/my/5c9189ce83f343d79...,1671040944_5c9189ce83f343d79ff5c619,배꼽집,별점5
97852,https://m.place.naver.com/my/5b9b3229c71f833d7...,1671040944_5b9b3229c71f833d7acfc1f7,배꼽집,별점5
97853,https://m.place.naver.com/my/5dd2183c8f87a842b...,1671040944_5dd2183c8f87a842bcb37f0c,배꼽집,별점5


In [141]:
a['ResCode'] = a['UserID'].apply(lambda x:x.split('_')[0])
a['UserID'] = a['UserID'].apply(lambda x:x.split('_')[1])
a['Rating'] = a['Rating'].apply(lambda x:x.split('별점')[1])

In [142]:
c['ResCode'] = c['UserID'].apply(lambda x:x.split('_')[0])
c['UserID'] = c['UserID'].apply(lambda x:x.split('_')[1])
c['Rating'] = c['Rating'].apply(lambda x:x.split('별점')[1])

In [144]:
d = pd.concat([a,b,c])

In [146]:
d = d.drop_duplicates()

In [147]:
d['ItemID'].nunique()

1366

In [148]:
concat

,user_url,UserID,ItemID,Rating,ResCode
53,https://m.place.naver.com/my/5dfb5c978f87a842b...,5dfb5c978f87a842bce18014,수미가샤브샤브,4.5,11847080
56,https://m.place.naver.com/my/5f048aef5a203ca38...,5f048aef5a203ca38fa3582a,수미가샤브샤브,5,11847080
57,https://m.place.naver.com/my/5e2425ea8f87a842b...,5e2425ea8f87a842bca5cecf,수미가샤브샤브,5,11847080
58,https://m.place.naver.com/my/5f94d593cadf2feab...,5f94d593cadf2feab37c9760,수미가샤브샤브,5,11847080
59,https://m.place.naver.com/my/5f5eb06b6c3d7f733...,5f5eb06b6c3d7f7334d4c883,수미가샤브샤브,4.5,11847080
...,...,...,...,...,...
338849,https://m.place.naver.com/my/5e31545b8f87a842b...,5e31545b8f87a842bcbe7ed5,맛쟁이떡볶이 본점,3.5,1328070159
338850,https://m.place.naver.com/my/5f1be18dca84d8e54...,5f1be18dca84d8e545e26bca,맛쟁이떡볶이 본점,5,1328070159
338851,https://m.place.naver.com/my/5f06efb7848333b0d...,5f06efb7848333b0d5c21d1b,맛쟁이떡볶이 본점,5,1328070159
338852,https://m.place.naver.com/my/5f1d3b049ec8258e4...,5f1d3b049ec8258e4a2c67ce,맛쟁이떡볶이 본점,4,1328070159


In [162]:
len(tmp)

748941

In [163]:
tmp = pd.concat([d, concat, rating_df])

In [164]:
tmp = tmp.drop_duplicates()

In [165]:
len(tmp)

755077

In [167]:
tmp['ItemID'].nunique()

2815

In [168]:
tmp['UserID'].nunique()

205400

In [156]:
tmp

,user_url,UserID,ItemID,Rating,ResCode
0,https://m.place.naver.com/my/5e1877e68f87a842b...,5e1877e68f87a842bc8a03ce,오늘와인한잔 양천향교역점,3,1049678083
1,https://m.place.naver.com/my/5efdf404854ca9293...,5efdf404854ca9293112e92c,오늘와인한잔 양천향교역점,5,1049678083
2,https://m.place.naver.com/my/5f44cb53a602e98f5...,5f44cb53a602e98f564c63ad,오늘와인한잔 양천향교역점,3,1049678083
3,https://m.place.naver.com/my/5dd91a419ec8258e4...,5dd91a419ec8258e4a4342bb,오늘와인한잔 양천향교역점,5,1049678083
4,https://m.place.naver.com/my/5efde995b1f7027e8...,5efde995b1f7027e8a6ca104,오늘와인한잔 양천향교역점,3,1049678083
...,...,...,...,...,...
338849,https://m.place.naver.com/my/5e31545b8f87a842b...,5e31545b8f87a842bcbe7ed5,맛쟁이떡볶이 본점,3.5,1328070159
338850,https://m.place.naver.com/my/5f1be18dca84d8e54...,5f1be18dca84d8e545e26bca,맛쟁이떡볶이 본점,5,1328070159
338851,https://m.place.naver.com/my/5f06efb7848333b0d...,5f06efb7848333b0d5c21d1b,맛쟁이떡볶이 본점,5,1328070159
338852,https://m.place.naver.com/my/5f1d3b049ec8258e4...,5f1d3b049ec8258e4a2c67ce,맛쟁이떡볶이 본점,4,1328070159


In [166]:
tmp.to_csv('scrapping_half.csv', index=False)

In [173]:
rating_df

,user_url,UserID,ItemID,Rating
210,https://m.place.naver.com/my/5efe7a55493ad9a9a...,1062855533_5efe7a55493ad9a9a0ebc811,청와옥 석촌호수직영점,별점5
215,https://m.place.naver.com/my/5ead866b8f87a842b...,1062855533_5ead866b8f87a842bcfb2a66,청와옥 석촌호수직영점,별점5
225,https://m.place.naver.com/my/5e9e75778f87a842b...,1062855533_5e9e75778f87a842bc2109f3,청와옥 석촌호수직영점,별점4
226,https://m.place.naver.com/my/5df001558f87a842b...,1062855533_5df001558f87a842bc74aff1,청와옥 석촌호수직영점,별점4
228,https://m.place.naver.com/my/5be903a649a1a12ec...,1062855533_5be903a649a1a12ecd108858,청와옥 석촌호수직영점,별점5
...,...,...,...,...
194147,https://m.place.naver.com/my/5c83d7d5463f840e7...,1651918728_5c83d7d5463f840e73ed5b4e,저스트텐동 연남본점,별점5
194148,https://m.place.naver.com/my/5b7af1ae9bbeb6c65...,1651918728_5b7af1ae9bbeb6c65dbffb51,저스트텐동 연남본점,별점4
194149,https://m.place.naver.com/my/5b6c50fe8f68a5fbc...,1651918728_5b6c50fe8f68a5fbc40b084f,저스트텐동 연남본점,별점5
194150,https://m.place.naver.com/my/5c1b830d4a9b9dd07...,1651918728_5c1b830d4a9b9dd073e42717,저스트텐동 연남본점,별점5


In [174]:
rating_df['ResCode'] = rating_df['UserID'].apply(lambda x:x.split('_')[0])
rating_df['UserID'] = rating_df['UserID'].apply(lambda x:x.split('_')[1])
rating_df['Rating'] = rating_df['Rating'].apply(lambda x:x.split('별점')[1])

In [175]:
rating_df = rating_df.drop_duplicates()

In [177]:
rating_df.to_csv('rating_df.csv', index=False)

In [178]:
rating_df

,user_url,UserID,ItemID,Rating,ResCode
210,https://m.place.naver.com/my/5efe7a55493ad9a9a...,5efe7a55493ad9a9a0ebc811,청와옥 석촌호수직영점,5,1062855533
215,https://m.place.naver.com/my/5ead866b8f87a842b...,5ead866b8f87a842bcfb2a66,청와옥 석촌호수직영점,5,1062855533
225,https://m.place.naver.com/my/5e9e75778f87a842b...,5e9e75778f87a842bc2109f3,청와옥 석촌호수직영점,4,1062855533
226,https://m.place.naver.com/my/5df001558f87a842b...,5df001558f87a842bc74aff1,청와옥 석촌호수직영점,4,1062855533
228,https://m.place.naver.com/my/5be903a649a1a12ec...,5be903a649a1a12ecd108858,청와옥 석촌호수직영점,5,1062855533
...,...,...,...,...,...
194147,https://m.place.naver.com/my/5c83d7d5463f840e7...,5c83d7d5463f840e73ed5b4e,저스트텐동 연남본점,5,1651918728
194148,https://m.place.naver.com/my/5b7af1ae9bbeb6c65...,5b7af1ae9bbeb6c65dbffb51,저스트텐동 연남본점,4,1651918728
194149,https://m.place.naver.com/my/5b6c50fe8f68a5fbc...,5b6c50fe8f68a5fbc40b084f,저스트텐동 연남본점,5,1651918728
194150,https://m.place.naver.com/my/5c1b830d4a9b9dd07...,5c1b830d4a9b9dd073e42717,저스트텐동 연남본점,5,1651918728
